# Avance de regresion

Se entrena un modelo con todas las variables que no tienen valor constante para predecir la parte del valor de venta.

**NOTA: Este notebook se hizo con un computador que tenia teclado en ingles, asi que por eso podrian hacer falta las tildes**

## Librerias

In [1]:
!pip install scikit-learn-intelex


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [2]:
%load_ext autoreload
%autoreload 2

import os
import gc
import re
import seaborn as sns

from sklearnex import patch_sklearn
patch_sklearn()

import numpy as np

import pandas as pd
from typing import Tuple, List, Dict, Any
from tqdm import tqdm

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import RandomizedSearchCV

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Constantes 

In [3]:
# Dataset file
DATA_FOLDER = "data"
# Test percentage
TEST_PERC = 0.2
# Random seed
RANDOM_STATE = 42

In [4]:
PERCENTILES = [0.05, 0.25, 0.5, 0.75, 0.95]
PERCENTILE_COLS = [f"percentile__{int(perc*100)}" for perc in PERCENTILES]

In [5]:
# Label
LABEL_COLUMN = "Parte de valor_venta"
# Feature columns
FEATURE_COLUMNS = [
    "Destino",
    "Tipo de vuelo",
    "Sala",
    #"Muelle" Right now there is only one
    "Pasajeros saliendo",
    "AEROLÍNEA",
    "Tipo aerolínea",
    "Hora entera",
    "FECHA DIA",
    "Semana",
    "Mes",
    "Día",
    "day_of_year"
]
# Scaler
SCALER = StandardScaler()

## Utils

In [6]:
def load_marcas():
    marca_files = [
        (
            file.split(".")[0],
            pd.read_parquet(f"{DATA_FOLDER}/{file}")
        )
        for file in os.listdir(DATA_FOLDER)
        if ("marca " in file) and (".parquet" in file)
    ]
    return dict(marca_files)

In [7]:
def hash_column(df: pd.DataFrame, col: str) -> Tuple[np.array, List[str]]:
    """Creates hashes for categorical col"""
    dimensions = int(np.log2(df[col].nunique()))
    col_hasher = FeatureHasher(
        dimensions,
        input_type="string"
    )
    hashed_values = col_hasher.fit_transform(
        df[col].values.reshape(-1, 1)
    ).todense()
    
    new_columns = [
        f"{col}_{x}" for x in range(dimensions)
    ]
    
    return hashed_values, new_columns

## Ejecucion

#### Carga de datos

In [8]:
marcas = load_marcas()

In [9]:
marca_features = {}
for marca, df_marca in marcas.items():
    print(marca)
    df_marca.info()
    df_marca["day_of_year"] = df_marca["STD"].dt.dayofyear
    marca_features[marca] = df_marca[FEATURE_COLUMNS].copy(deep=True)

marca 3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43246 entries, 0 to 43245
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Destino                      43246 non-null  object        
 1   Tipo de vuelo                43246 non-null  object        
 2   Sala                         43246 non-null  object        
 3   Muelle                       43246 non-null  object        
 4   AEROLÍNEA                    43246 non-null  object        
 5   Tipo aerolínea               43246 non-null  object        
 6   STD                          43246 non-null  datetime64[ns]
 7   Hora entera                  43246 non-null  int64         
 8   FECHA DIA                    43246 non-null  object        
 9   Pasajeros saliendo           43246 non-null  float64       
 10  Semana                       43246 non-null  int64         
 11  Mes                          4324

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44026 entries, 0 to 44025
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Destino                      44026 non-null  object        
 1   Tipo de vuelo                44026 non-null  object        
 2   Sala                         44026 non-null  object        
 3   Muelle                       44026 non-null  object        
 4   AEROLÍNEA                    44026 non-null  object        
 5   Tipo aerolínea               44026 non-null  object        
 6   STD                          44026 non-null  datetime64[ns]
 7   Hora entera                  44026 non-null  int64         
 8   FECHA DIA                    44026 non-null  object        
 9   Pasajeros saliendo           44026 non-null  float64       
 10  Semana                       44026 non-null  int64         
 11  Mes                          44026 non-nu

### Separacion de features numericas y categoricas

In [10]:
NUMERICAL_FEATURES = marca_features["marca 1"].select_dtypes(include=np.number).columns
CATEGORICAL_FEATURES = marca_features["marca 1"].select_dtypes(include="O").columns

In [11]:
NUMERICAL_FEATURES

Index(['Pasajeros saliendo', 'Hora entera', 'Semana', 'Mes', 'Día',
       'day_of_year'],
      dtype='object')

In [12]:
CATEGORICAL_FEATURES

Index(['Destino', 'Tipo de vuelo', 'Sala', 'AEROLÍNEA', 'Tipo aerolínea',
       'FECHA DIA'],
      dtype='object')

### Numero de categorias por cada feature categorica

In [13]:
for marca, df_marca in marcas.items():
    print("*" * 5, marca, "*" * 5)
    for col in CATEGORICAL_FEATURES:
        print(col, df_marca[col].nunique())

***** marca 3 *****
Destino 59
Tipo de vuelo 1
Sala 18
AEROLÍNEA 37
Tipo aerolínea 4
FECHA DIA 7
***** marca 6 *****
Destino 59
Tipo de vuelo 1
Sala 19
AEROLÍNEA 38
Tipo aerolínea 4
FECHA DIA 7
***** marca 1 *****
Destino 62
Tipo de vuelo 1
Sala 19
AEROLÍNEA 39
Tipo aerolínea 4
FECHA DIA 7
***** marca 2 *****
Destino 61
Tipo de vuelo 1
Sala 19
AEROLÍNEA 38
Tipo aerolínea 4
FECHA DIA 7
***** marca 10 *****
Destino 53
Tipo de vuelo 1
Sala 18
AEROLÍNEA 34
Tipo aerolínea 4
FECHA DIA 7
***** marca 5 *****
Destino 59
Tipo de vuelo 1
Sala 18
AEROLÍNEA 38
Tipo aerolínea 4
FECHA DIA 7
***** marca 7 *****
Destino 60
Tipo de vuelo 1
Sala 19
AEROLÍNEA 38
Tipo aerolínea 4
FECHA DIA 7
***** marca 9 *****
Destino 59
Tipo de vuelo 1
Sala 19
AEROLÍNEA 38
Tipo aerolínea 4
FECHA DIA 7


### Codificando features categoricas
Se usa onehot para features con bajo numero de categorias y hashing para el resto.

In [14]:
marca_categorical_features = {}
for marca, df_marca in marcas.items():
    df_cat_features = pd.concat([
            pd.get_dummies(df_marca["Tipo aerolínea"]),
            pd.get_dummies(df_marca["FECHA DIA"])
        ],
        axis=1
    )
    high_dim_dfs = []
    for col in ["Destino", "Sala", "AEROLÍNEA"]:
        col_hash, col_subcols = hash_column(df_marca, col)
        df_col_hash = pd.DataFrame(col_hash)
        df_col_hash.columns = col_subcols

        high_dim_dfs.append(df_col_hash)

    df_high_dim_dfs = pd.concat(
        high_dim_dfs,
        axis=1
    )
    df_cat_features = pd.concat([
        df_cat_features,
        df_high_dim_dfs
    ],
    axis=1)
    marca_categorical_features[marca] = df_cat_features

### Anadiendo features numericas

In [15]:
marca_numerical_features = {}
for marca, df_marca in marcas.items():
    marca_numerical_features[marca] = df_marca[NUMERICAL_FEATURES]

In [16]:
marca_numerical_features.keys()

dict_keys(['marca 3', 'marca 6', 'marca 1', 'marca 2', 'marca 10', 'marca 5', 'marca 7', 'marca 9'])

In [17]:
marca_categorical_features.keys()

dict_keys(['marca 3', 'marca 6', 'marca 1', 'marca 2', 'marca 10', 'marca 5', 'marca 7', 'marca 9'])

In [18]:
marca_features = {}
for marca, df_numerical_marca in marca_numerical_features.items():
    marca_features[marca] = {}
    marca_features[marca]["features"] = pd.concat(
        [
            df_numerical_marca,
            marca_categorical_features[marca]
        ],
        axis=1
    )
    marca_features[marca]["labels"] = marcas[marca][LABEL_COLUMN].values

### Train test split and scaling

In [19]:
marca_features.keys()

dict_keys(['marca 3', 'marca 6', 'marca 1', 'marca 2', 'marca 10', 'marca 5', 'marca 7', 'marca 9'])

In [20]:
marca_splits = {}
for marca, marca_data in marca_features.items():
    X_marca_features = marca_data["features"]
    y_marca_labels = marca_data["labels"]
    X_train, X_test, y_train, y_test = train_test_split(
        X_marca_features,
        y_marca_labels,
        test_size=TEST_PERC,
        random_state=RANDOM_STATE
    )
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    marca_splits[marca] = {}
    marca_splits[marca]["X_train"] = X_train
    marca_splits[marca]["X_test"] = X_test
    marca_splits[marca]["y_train"] = y_train
    marca_splits[marca]["y_test"] = y_test
    marca_splits[marca]["scaler"] = scaler

### Creacion de modelo y entrenamiento

Se usa randomized search cv para explorar hiperparametros y un randomforestregressor debido a la cantidad de columnas categoricas.

In [21]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [22]:
for  marca, marca_data in tqdm(marca_splits.items()):
    # Use the random grid to search for best hyperparameters
    # First create the base model to tune
    rf = RandomForestRegressor(n_jobs=-1)
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    rf_random = RandomizedSearchCV(
        estimator = rf,
        param_distributions = random_grid,
        n_iter = 100,
        cv = 3,
        verbose=2,
        random_state=RANDOM_STATE,
        n_jobs = -1,
        scoring="neg_root_mean_squared_error"
    )# Fit the random search model
    rf_random.fit(marca_data["X_train"], marca_data["y_train"])
    marca_data["model"] = rf_random.best_estimator_

  0%|                                                     | 0/8 [00:00<?, ?it/s]

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time=  29.1s


/home/johan/miniconda3/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time=  28.9s
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=False, max_depth=100, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1000; total time= 1.0min
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=  18.7s
[CV] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=600; total time=  55.0s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=  20.2s
[CV] END bootstrap=False, max_depth

[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=2000; total time=  47.0s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1600; total time= 1.1min
[CV] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1600; total time= 3.1min
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=1800; total time=   0.0s
[CV] END bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1400; total time= 2.8min
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1600; total time= 1.2min
[CV] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1600; total time= 3.1min
[CV] END bootstrap=False, max_dept

...

[CV] END bootstrap=True, max_depth=60, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1800; total time= 1.5min
[CV] END bootstrap=True, max_depth=40, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=800; total time=  30.9s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=  15.6s
[CV] END bootstrap=False, max_depth=50, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=   0.0s
[CV] END bootstrap=False, max_depth=50, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=   0.0s
[CV] END bootstrap=False, max_depth=50, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=   0.0s
[CV] END bootstrap=False, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=1800; total time=   0.0s
[CV] END bootstrap=False, max_depth=1

[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=800; total time=  34.1s
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=400; total time=   9.8s
[CV] END bootstrap=True, max_depth=80, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=80, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=80, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.0s
[CV] END bootstrap=False, max_depth=60, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=False, max_depth=60, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=False, max_depth=60, m

/home/johan/miniconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
123 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
62 fits failed with the following error:
Traceback (most recent call last):
  File "/home/johan/miniconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/johan/miniconda3/lib/python3.10/site-packages/daal4py/sklearn/_device_offload.py", line 88, in wrapper_with_self
    return wrapper_impl(self, *args, **kwargs)
  File "/home/johan/miniconda3/lib/python3.10/site-packages/daal4py/sklearn/_device_offload.py", line 74, in wrapper

[CV] END bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1800; total time= 1.2min
[CV] END bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=600; total time=  37.5s
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=  12.6s
[CV] END bootstrap=False, max_depth=60, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=False, max_depth=60, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1400; total time=  51.2s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1000; total time=  16.9s
[CV] END bootstrap=False, max_depth=

100%|██████████████████████████████████████████| 8/8 [1:10:10<00:00, 526.37s/it]


### Evaluacion del modelo

In [23]:
all_metrics = []
for marca, marca_data in tqdm(marca_splits.items()):
    model = marca_data["model"]
    X_train = marca_data["X_train"]
    X_test = marca_data["X_test"]
    y_train = marca_data["y_train"]
    y_test = marca_data["y_test"]
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    
    df_metrics = pd.DataFrame({
        "marca": marca,
        "MAE_train": [mean_absolute_error(y_train, y_pred_train)],
        "MAE_test": [mean_absolute_error(y_test, y_pred_test)],
        "MSE_train": [mean_squared_error(y_train, y_pred_train)],
        "MSE_test": [mean_squared_error(y_test, y_pred_test)],
        "RMSE_train": [np.sqrt(mean_squared_error(y_train, y_pred_train))],
        "RMSE_test": [np.sqrt(mean_squared_error(y_test, y_pred_test))],
        "R2_train": [r2_score(y_train, y_pred_train)],
        "R2_test": [r2_score(y_test, y_pred_test)],
        "MAPE_train": [mean_absolute_percentage_error(y_train, y_pred_train)],
        "MAPE_test": [mean_absolute_percentage_error(y_test, y_pred_test)],
    })
    
    df_percentiles = pd.DataFrame(
        marcas[marca]["Parte de valor_venta"].quantile(PERCENTILES).tolist(),
        index=PERCENTILE_COLS
    ).T
    
    df_metrics = pd.concat(
        [
            df_metrics,
            df_percentiles
        ],
        axis=1
    )
    
    all_metrics.append(df_metrics)
df_all_metrics = pd.concat(all_metrics)

100%|█████████████████████████████████████████████| 8/8 [00:34<00:00,  4.31s/it]


In [24]:
df_all_metrics

,marca,MAE_train,MAE_test,MSE_train,MSE_test,RMSE_train,RMSE_test,R2_train,R2_test,MAPE_train,MAPE_test,percentile__5,percentile__25,percentile__50,percentile__75,percentile__95
0,marca 3,7.324074e+06,3.186517e+07,9.827116e+13,1.680737e+15,9.913181e+06,4.099679e+07,0.972208,0.522790,7.486539e+19,6.209865e+20,1.696171e+07,4.811064e+07,85270678.2,1.325707e+08,2.103328e+08
0,marca 6,4.505879e+06,1.948821e+07,3.666534e+13,6.248312e+14,6.055192e+06,2.499662e+07,0.970973,0.500659,5.152800e+19,2.463388e+20,5.975200e+06,2.032657e+07,40192500.0,6.954220e+07,1.220374e+08
0,marca 1,2.050690e+07,4.345206e+07,8.281991e+14,3.718563e+15,2.877845e+07,6.098002e+07,0.938295,0.719648,1.313495e+21,1.639098e+21,1.877772e+07,6.933650e+07,137081760.0,2.425796e+08,3.812926e+08
0,marca 2,4.180386e+06,1.776829e+07,3.229844e+13,5.314450e+14,5.683172e+06,2.305309e+07,0.973368,0.551155,5.059754e+19,3.345595e+20,8.645520e+06,2.672000e+07,48760000.0,7.761864e+07,1.192933e+08
0,marca 10,1.580686e+05,4.479904e+06,7.308839e+10,3.494768e+13,2.703487e+05,5.911656e+06,0.998811,0.415985,1.222330e+18,1.079204e+20,1.760000e+06,3.998000e+06,7886400.0,1.429942e+07,2.954780e+07
0,marca 5,2.857354e+06,1.241169e+07,1.492181e+13,2.521824e+14,3.862876e+06,1.588025e+07,0.964128,0.391700,2.329609e+19,1.463027e+20,1.075314e+06,7.912271e+06,17660000.0,3.496000e+07,7.088000e+07
0,marca 7,1.057922e+06,4.565795e+06,2.096932e+12,3.490067e+13,1.448079e+06,5.907679e+06,0.964353,0.405842,4.094267e+20,1.999775e+21,0.000000e+00,2.500889e+06,6778991.6,1.336807e+07,2.441345e+07
0,marca 9,2.839920e+06,5.882332e+06,1.325451e+13,5.676934e+13,3.640675e+06,7.534543e+06,0.871289,0.448409,3.771383e+19,1.209963e+20,4.246060e+06,1.070086e+07,17583060.0,2.540246e+07,3.703675e+07


In [25]:
df_all_metrics.to_parquet("data/marca_metrics.parquet", index=False)

[CV] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time=  44.5s
[CV] END bootstrap=True, max_depth=80, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1600; total time=  57.5s
[CV] END bootstrap=True, max_depth=40, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=True, max_depth=40, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=True, max_depth=40, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=1600; total time= 1.2min
[CV] END bootstrap=True, max_depth=110, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=600; total time=  23.2s
[CV] END bootstrap=True, max_depth

[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=2000; total time=  47.3s


## Conclusiones
- El error RMSE esta en el mismo nivel de magnitud que la mediana de los valores de Parte de valor_venta en test, mas especificamente un valor cercano a un tercio de la mediana, asi que se requiere refinamiento de datos y modelo.